In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!cp drive/MyDrive/MMDS-data/spark-3.1.1-bin-hadoop3.2.tgz .
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

cp: cannot stat 'drive/MyDrive/MMDS-data/spark-3.1.1-bin-hadoop3.2.tgz': No such file or directory


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark as spark
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, col, lower, regexp_replace , monotonically_increasing_id
from pyspark.ml.linalg import SparseVector
from pyspark.sql.types import StructType, StructField, StringType , ArrayType ,IntegerType

from pyspark.ml.linalg import SparseVector
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_set ,array_distinct , explode,when  , array_contains ,collect_list,  udf
import random
import hashlib
import sys
import itertools
import time

sc = SparkContext('local', 'First App')
sc = SparkContext.getOrCreate('local')
spark = SQLContext(sc)

In [ ]:
class LargDataMinHashLSH:
    def __init__(self, path):
        self.documents = None
        self.path =  path
        self.kshingles = 8
        self.shingles = None
        self.numberOfVectorSignature = 100
        self.lensignatrue = None
        self.df_shingles = None
        self.shingle_ = None
        self.hash_func = None
        self.signatures = None
        self.vectorboolean = None


    def shingling(self, dataframe):
      k = self.kshingles
      rdd = dataframe.rdd

      # Tạo RDD chứa các shingles
      shingles_rdd = rdd.map(lambda shingle: (shingle[0], [shingle[1][i:i+k] for i in range(len(shingle[1]) - k + 1)]))
      shingles_rdd_beforeJoin = shingles_rdd.map( lambda x : (0 , (str(x[0]) , x[1])))


      # Lấy các shingle duy nhất
      unique_shingles_rdd = shingles_rdd \
                  .flatMap(lambda x: x[1]) \
                  .distinct() \
                  .map( lambda x : ( 0 , [x])) \
                  .sortBy(lambda x: x[1])\
                  .reduceByKey( lambda x , y :  x +  y)

      self.shingle_  = unique_shingles_rdd
      # Kết hợp các RDD
      joined_rdd = shingles_rdd_beforeJoin.join(unique_shingles_rdd)
      # Tạo DataFrame từ kết quả

      result_rdd = joined_rdd.map(lambda x: (x[1][0][0],   [1 if x[1][1][i] in x[1][0][1] else 0 for i in range(len(x[1][1]))]))
      selected_df =result_rdd.toDF(["docid", "features"])
      return selected_df

    def minhashing(self  ,  df) :
      count = len(self.shingle_.first()[1])
      n = 100
      # Tạo một danh sách hoán vị ngẫu nhiên cho mỗi hoán vị
      permutations  = sc.parallelize(range(0 , n))\
        .map(lambda x: ( 0  , [random.sample(list(range(1, count + 1)), count)])) \
        .reduceByKey(lambda x , y :  x+y)

      self.hash_func = permutations

      # Kết hợp DataFrame với danh sách hoán vị
      joined_rdd =  df.rdd.cartesian(permutations)
      joined_rdd =   joined_rdd.map(lambda x :   (x[0][0] , [ min(perm_val for perm_val, row_val in zip(perm, x[0][1]) if row_val) for perm in x[1][1]]))
      self.signatures = joined_rdd
      signatures_df = spark.createDataFrame(joined_rdd , schema=["docid" , "hashes"])
      return signatures_df

    def locality_sensitivity_hashing(self, df):
        #Tạo các band
        signature = df.rdd
        b=20 # Số band
        r=5 # Số hàng trong mỗi band

        # create band
        band_lsh =  signature.map(lambda band :  (band[0] , [[band[1][i] for i in range(j* r , j*r  + r) ] for j in range(b)]))

        # hash band
        hashed_bands = band_lsh.flatMap(lambda band: [(minHash(tuple(band[1][j]), j), band[0]) for j in range(b)])

        # build bucket
        buckets = hashed_bands.map(lambda bucket : (bucket[0] , bucket[1]))\
                              .reduceByKey(lambda doc1 , doc2 : doc1 +  doc2)\
                              .filter(lambda collision : len(collision [1]) > 1)

        # Chuyển đổi kết quả thành DataFrame
        res = buckets.flatMap(lambda x : [( i,x[0]  )for i in x[1]])
        res_df = spark.createDataFrame(res , schema=["iddoc"  , "bucket_id"])
        return res_df

    def approxNearestNeighbors(self , key : str , n :int):
        """
            step 1 : convert str -> boolean -> signature
            step 2 : loop each documents  distance item with each document use Jaccard
            step 3 : take(n)
        """
        # Bước 1: Chuyển đổi chuỗi thành vector boolean và chữ ký
        # shingles_rdd = self.df_shingles.rdd
        k = self.kshingles
        union_shingle = self.shingle_
        shingles_key = [(0 , [key[i:i+k]]) for i in range(len(key) - k + 1)]
        querry_doc_rdd = sc.parallelize(shingles_key).reduceByKey(lambda x, y: x + y)
        joined_rdd =  querry_doc_rdd.cartesian(union_shingle)
        querry_doc_rdd = joined_rdd.map(lambda x: (0 , [1 if x[1][1][i] in x[0][1] else 0 for i in range(len(x[1][1]))]))

        # Chuyển đổi thành (signature)
        permutations = self.hash_func
        joined_rdd =  querry_doc_rdd.cartesian(permutations)
        querry_doc_rdd_signature =   joined_rdd.map(lambda x :   (x[0][0] , [ min(perm_val for perm_val, row_val in zip(perm, x[0][1]) if row_val) for perm in x[1][1]]))
        # querry_doc_rdd_signature.toDF().show(truncate = False)

        # Tính Jaccard similarity cho mỗi cặp
        # cartesian_product = shingles_rdd.cartesian(querry_doc_rdd_signature)

        signatures =  self.signatures
        cartesian_product = signatures.cartesian(querry_doc_rdd_signature)
        # cartesian_product.toDF().show(truncate=False)

        #Compute Jaccard similarity for each pair
        jaccard = cartesian_product.map(lambda x : (  x[0][0]  , float(len(set(x[0][1]).intersection(x[1][1]))) /  float(len(set(x[0][1]).union(x[1][1]))) ))\
                                  .sortBy(lambda x : x[1] , ascending= False)\
                                  .take(n)

        # Tạo DataFrame từ kết quả
        jaccard_ok =  spark.createDataFrame(jaccard , schema=["docid", "distance"])
        jaccard_ok.show()


    def run(self):
        self.documents = loadDataFrame(self.path)
        start_time = time.time()
        step1  = self.shingling(self.documents)
        # step1.show(10 , truncate = False)
        end_time = time.time()
        print(f"Execution time: { start_time -  end_time} seconds")

        start_time = time.time()
        step2 = self.minhashing(step1)
        # step2.show(10  ,truncate = False)
        end_time = time.time()
        print(f"Execution time: { start_time -  end_time} seconds")

        start_time = time.time()
        step3 = self.locality_sensitivity_hashing(step2)
        # step3.show(10)
        end_time = time.time()
        print(f"Execution time: { start_time -  end_time} seconds")

def loadDataFrame(path: str):
    try:
        spark = SparkSession.builder.appName("LSH").getOrCreate()
        rdd = sc.textFile(path)
        schema = StructType([
            StructField("id", StringType(), nullable=True),
            StructField("features", StringType(), nullable=True)
        ])
        df = spark.createDataFrame(rdd.zipWithIndex().map(lambda line: (str(line[1]) , line[0])) , schema=schema)
        return df
    except Exception as e:
        print('Error:', e)
        return None

def hashFamily(i):
  resultSize = 8
  # how many bytes we want back
  maxLen = 20
  # how long can our i be (in decimal)
  salt = str(i).zfill(maxLen)[-maxLen:]
  def hashMember(x):
    sequence = x + salt
    return hashlib.sha1(sequence.encode("utf-8")).digest()[-resultSize:]
  return hashMember

#Given a list of shingles, compute the hash for every shingle and keep the smallest value
def minHash(shingles:list, i:int) -> int:
  hash_fn = hashFamily(i)
  minSignature = float('inf') #big number
  for s in shingles:
    #some casts are needed
    hashSignature = int.from_bytes(hash_fn(str(s)), sys.byteorder)
    if hashSignature < minSignature:
        minSignature = hashSignature
  return minSignature

#Get the pairs from a list
def get_pairs(collisions_list:list) -> tuple:
  pair_list=[]
  for pair in itertools.combinations(collisions_list,2):
    pair_list.append(pair)
  return tuple(pair_list)


path = '/content/WebOfScience-50.txt'
lsh = LargDataMinHashLSH(path)
lsh.run()


querry = '1,2-Dichloropropane (1,2-DCP) and dichloromethane (DCM) are possible causative agents associated with the development of cholangiocarcinoma in employees working in printing plant in Osaka, Japan. However, few reports have demonstrated an association between these agents and cholangiocarcinoma in rodent carcinogenicity studies. Moreover, the combined effects of these compounds have not been fully elucidated. In the present study, we evaluated the in vivo mutagenicity of 1,2-DCP and DCM, alone or combined, in the livers of gpt delta rats. Six-week-old male F344 gpt delta rats were treated with 1,2-DCP, DCM or 1,2-DCP+DCM by oral administration for 4weeks at the dose (200mgkg(-1) body weight 1,2-DCP and 500mgkg(-1) body weight DCM) used in the carcinogenesis study performed by the National Toxicology Program. In vivo mutagenicity was analyzed by gpt mutation/Spi(-) assays in the livers of rats. In addition, gene and protein expression of CYP2E1 and GSTT1, the major enzymes responsible for the genotoxic effects of 1,2-DCP and DCM, were analyzed by quantitative polymerase chain reaction and western blotting. Gpt and Spi(-) mutation frequencies were not increased by 1,2-DCP and/or DCM in any group. Additionally, there were no significant changes in the gene and protein expression of CYP2E1 and GSTT1 in any group. These results indicated that 1,2-DCP, DCM and 1,2-DCP+DCM had no significant impact on mutagenicity in the livers of gpt delta rats under our experimental conditions. Copyright (c) 2016 John Wiley & Sons, Ltd.'
lsh.approxNearestNeighbors(querry  , 5)


Execution time: -1176.1602883338928 seconds
Execution time: -14.308485507965088 seconds
Execution time: -91.87277722358704 seconds
+-----+------------------+
|docid|          distance|
+-----+------------------+
|    3|               1.0|
|   15|0.5466666666666666|
|   29|0.5128205128205128|
|    1|               0.5|
|   40|               0.5|
+-----+------------------+

